## ewf-ext-02-03-04 - NDVI anomalies of growing season time series per pixel

NDVI anomalies of growing season time series per pixel

---

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'NDVI anomalies of growing season time series per pixel'),
                ('abstract', 'NDVI anomalies of growing season time series per pixel'),
                ('id', 'ewf-ext-02-03-04')])

### <a name="parameter">Parameter Definition 

In [ ]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON ((-8.864205 38.88616500000001, -8.864205 38.986165, -8.964205000000002 38.986165, -8.964205000000002 38.88616500000001, -8.864205 38.88616500000001))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

In [ ]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', 'P001'),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

In [ ]:
indexAndApiKeys = dict([('id', 'indexAndApiKeys'),
                        ('value', ''),
                        ('title', 'index,apikey pairs'),
                        ('abstract', 'index,apikey pairs'),
                        ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

Agg and LTA

In [ ]:
input_identifiers = ('60C6662B91CEB188007C3809F852EC0FB0378E2E','1D976D213B24ABFA2CAA46870019E296CE47A06E')

**Input references**

Agg and LTA

In [ ]:
input_references = ('https://catalog.terradue.com/better-ext-02-03-02/search?format=atom&uid=60C6662B91CEB188007C3809F852EC0FB0378E2E', 'https://catalog.terradue.com/better-ext-02-03-03/search?format=atom&uid=1D976D213B24ABFA2CAA46870019E296CE47A06E')

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = ""

#### Aux folders

In [ ]:
output_folder = ''

#### Import Modules

In [ ]:
import os

import numpy as np
from osgeo import gdal, ogr, osr

import datetime

import pandas as pd
import geopandas as gpd

import cioppy
ciop = cioppy.Cioppy()

#### Auxiliary vars

In [ ]:
check_results = True

#### Auxiliary methods

In [ ]:
# get metadata from catalog
def get_input_metadata (input_refs, apikeys):
    
    # for each product get metadata
    Result_Prod = []
    
    for index,product_ref in enumerate(input_refs):
        
        for index in apikeys:
            if index in product_ref:
                cat_index = index
                cat_apikey = apikeys[index]
        
        # since the search is by identifier
        Result_Prod.append(ciop.search(end_point = product_ref,params =[],output_fields='self,identifier,startdate,enclosure,startdate,enddate,wkt,title',creds='{}:{}'.format(cat_index,cat_apikey))[0] )
    

    input_metadata = gpd.GeoDataFrame.from_dict(Result_Prod)

    input_metadata['startdate'] = pd.to_datetime(input_metadata['startdate'])
    input_metadata['enddate'] = pd.to_datetime(input_metadata['enddate'])
    
    return input_metadata

    
def get_metadata(filepath):

    ds = gdal.Open(filepath)
    projection = ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    no_data_value = ds.GetRasterBand(1).GetNoDataValue()
    data_type = ds.GetRasterBand(1).DataType
    return projection, geotransform, no_data_value, data_type


def get_matrix_list(image_list):
    mat_list = []
    for img in image_list:
        dataset = gdal.Open(img)
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
        
        print(type(product_array))
    return mat_list


def get_matrix_list(image_list, mask_value = None):
    mat_list = []
    for img in image_list:
        dataset = gdal.Open(img)
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        
        if mask_value is not None:
             product_array = np.ma.masked_values (product_array, mask_value)
        
        mat_list.append(product_array)
        dataset = None
    return mat_list



def calc_anomaly(agg_file, LTA_file):
    
    
    if agg_file and LTA_file:
        
        
        agg_and_LTA = get_matrix_list([agg_file, LTA_file], -9999.)
        print('Aggregation and LTA converted to matrices')
                        
        anomaly_values = np.divide(agg_and_LTA[0] * 1.0, agg_and_LTA[1] * 1.0, out=np.zeros(agg_and_LTA[0].shape) - 9999.0, where=np.logical_and(agg_and_LTA[1]!=0, agg_and_LTA[0] > -9998, agg_and_LTA[1] > -9998) )
        
        anomaly_values = np.ma.masked_values (anomaly_values, -9999.0)
        
        
        projection, geotransform, no_data_value, data_type = get_metadata(agg_file)
                
        return anomaly_values, projection, geotransform, no_data_value, data_type
    
    else:
        return None, None, None


def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    if mask is not None and mask is not 0:
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)

    

def write_anomaly_output(anomaly, output_folder, mission_name, stats_name, first_date, last_date, lta_start_year, lta_end_year, mask_no_value, regionOfInterest, roi_name, projection, geo_transform, no_data_value):
    
    stats_period = first_date.strftime('%Y%j') + '_' + last_date.strftime('%Y%j')
    LTA_period = str(lta_start_year) + '-' + str(lta_end_year)
    
    filename = '_'.join(['Anomaly', mission_name, stats_name, roi_name, stats_period, 'LTA', LTA_period])
    
    filename = os.path.join(output_folder, filename + '.tif')
    
    write_output_image(filename, anomaly, 'GTiff', gdal.GDT_Float32, mask_no_value, projection, geo_transform, no_data_value)
    
    return filename


def get_formatted_date(date_str):
    date = datetime.datetime.strftime(date_str, '%Y-%m-%dT00:00:00Z')
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    first_date = get_formatted_date(first_date)
    last_date = get_formatted_date(last_date)
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

#### Auxiliary folders

In [ ]:
if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

#### Workflow

In [ ]:
message = 'Getting metadata from catalog' 
ciop.log('INFO', message)


# organize indexes and apikeys in a python dictionary
indexAndApiKeys_splited = indexAndApiKeys['value'].split(',')
apikeys = {}
for idx,ele in enumerate(indexAndApiKeys_splited):
    
    if (idx % 2 == 0):
        print(ele)
        apikeys[ele] = indexAndApiKeys_splited[idx+1]
        

# get input data from catalog
input_metadata = get_input_metadata (input_references, apikeys)


input_metadata_LTA = input_metadata[input_metadata['title'].str.find('LTA') != -1]
input_metadata_Agg = input_metadata[input_metadata['title'].str.find('LTA') == -1]


In [ ]:
input_metadata_LTA

In [ ]:
input_metadata_Agg

In [ ]:
# get file paths
filepath_agg = os.path.join(data_path, input_metadata_Agg['enclosure'].iloc[0].split('/')[-1])
filepath_LTA = os.path.join(data_path, input_metadata_LTA['enclosure'].iloc[0].split('/')[-1])

print(filepath_agg,filepath_LTA)

In [ ]:

if isinstance(input_identifiers, str):
    input_identifiers = [input_identifiers]

region_of_interest = regionOfInterest['value']
name_of_region = nameOfRegion['value']


# Agg

file_name_elements = os.path.basename(filepath_agg).split('.')[0].split('_')
misson_name = file_name_elements[0]
stats_name = file_name_elements[1]
aoi_name = file_name_elements[2]

print(misson_name, stats_name, aoi_name)


first_date = datetime.datetime.strptime(file_name_elements[3], "%Y%j")
last_date = datetime.datetime.strptime(file_name_elements[4], "%Y%j")

print(first_date)
print(last_date)


# LTA

file_name_elements = os.path.basename(filepath_LTA).split('.')[0].split('_')

start_year = file_name_elements[5].split('-')[0]
end_year = file_name_elements[-1].split('-')[1]

print(file_name_elements)

stats_name_LTA = file_name_elements[2]


print(start_year)
print(end_year)

print(filepath_agg)
print(filepath_LTA)        

message = 'Computing Anomaly' 
ciop.log('INFO', message)
    
    
anomaly_values, projection, geotransform, no_data_value, data_type = calc_anomaly(filepath_agg, filepath_LTA)



message = 'Writing anomaly image' 
ciop.log('INFO', message)

filename = write_anomaly_output(anomaly_values.data, output_folder, misson_name, stats_name, first_date, last_date, start_year, end_year, None, region_of_interest, name_of_region, projection, geotransform, no_data_value)

print(filename)

write_properties_file(filename, first_date, last_date, region_of_interest)

